## Creative Content Assisted by Generative AI using Amazon SageMaker: Inpainting Fill and Outpainting

The first example is called **inpainting fill**. It is the process of replacing a portion of an image with synthesized content based on a textual prompt. We will accomplish this using Stable Diffusion XL(SDXL) model from Amazon Bedrock.

The workflow is to provide the model with three inputs:

- A mask image that outlines the portion to be replaced
- A textual prompt describing the desired contents
- The original image

Bedrock can then produce a new image that replaces the masked area with the object, subject, or environment described in the prompt.

You can follow this example [here](../inpainting_eraser/) to host a Segment Anything Model (SAM) and generate the masks, but we also provided a [mask image](data/mask.png) you can use out of the box for this example.

**Note:** The mask image must have the same resolution and aspect ratio as the image being inpainted upon.

### Setup

In [ ]:
!pip install -Uq sagemaker
!pip install -Uq diffusers

In [ ]:
# Python Built-Ins:
import base64
import io
import json
import os
import sys

# External Dependencies:
import boto3
from PIL import Image
from IPython.display import display
from diffusers.utils import load_image, make_image_grid

bedrock_runtime = boto3.client(service_name="bedrock-runtime")

We will send the image to Bedrock API in base64 encoding, so first let's prepare that. Here is the function that will convert a Pillow image to base64.

In [ ]:
def image_to_base64(img) -> str:
    """Convert a PIL Image or local image file path to a base64 string for Amazon Bedrock"""
    if isinstance(img, str):
        if os.path.isfile(img):
            with open(img, "rb") as f:
                return base64.b64encode(f.read()).decode("utf-8")
        else:
            raise FileNotFoundError(f"File {img} does not exist")
    elif isinstance(img, Image.Image):
        buffer = io.BytesIO()
        img.save(buffer, format="PNG")
        return base64.b64encode(buffer.getvalue()).decode("utf-8")
    else:
        raise ValueError(f"Expected str (filename) or PIL Image. Got {type(img)}")

### Download the initial image & mask

In [ ]:
image = load_image(
    "https://hf.co/datasets/huggingface/documentation-images/resolve/main/diffusers/input_image_vermeer.png"
)
mask = Image.open('data/mask.jpg')

make_image_grid([image, mask], rows=1, cols=2)

### Use Stable Diffusion XL 1.0 from Bedrock to inpaint the image

In [ ]:
inpaint_prompt = "The Mona Lisa wearing a wig"
style_preset = "digital-art"  # (e.g. photographic, digital-art, cinematic, ...)

In [ ]:
request = json.dumps({
    "text_prompts":[{"text": inpaint_prompt}],
    "init_image": image_to_base64(image),
    "mask_source": "MASK_IMAGE_WHITE",
    "mask_image": image_to_base64(mask),
    "cfg_scale": 10,
    "seed": 10,
    "style_preset": style_preset,
})
modelId = "stability.stable-diffusion-xl"

response = bedrock_runtime.invoke_model(body=request, modelId=modelId)
response_body = json.loads(response.get("body").read())

image_2_b64_str = response_body["artifacts"][0].get("base64")
inpaint = Image.open(io.BytesIO(base64.decodebytes(bytes(image_2_b64_str, "utf-8"))))

make_image_grid([image, inpaint], rows=1, cols=2)

The second example is called **outpainting**. It is the process technique that extends or extrapolates beyond the original image borders. We will accomplish this using Titan Image Generator from Amazon Bedrock.

The workflow is to provide the model with three inputs:

- Extending the canvas of orginal image
- creating an mask of extended area
- A textual prompt describing the desired contents

Titan Image generator can fill the extending area according to the textual prompt. This is very useful in situation will you need to change aspect ratio of the image, expanding repetitive textures, or expanding the scope of a scene by filling additional space and objects.

### Extending the original image

In [ ]:
original_width, original_height = image.size

target_width = 1024 #extended canvas size
target_height = 1024
position = ( #position the existing image in the center of the larger canvas
    int((target_width - original_width) * 0.5), 
    int((target_height - original_height) * 0.5),
)

extended_image = Image.new("RGB", (target_width, target_height), (235, 235, 235))
extended_image.paste(image, position)

# create a mask of the extended area
inside_color_value = (0, 0, 0) #inside is black - this is the masked area
outside_color_value = (255, 255, 255)

mask_image = Image.new("RGB", (target_width, target_height), outside_color_value)
original_image_shape = Image.new(
    "RGB", (original_width-40, original_height-40), inside_color_value
)
mask_image.paste(original_image_shape, tuple(x+20 for x in position))
make_image_grid([extended_image, mask_image], rows=1, cols=2)

## Use Titan Image Generator from Bedrock to extend the image

In [ ]:
# Configure the inference parameters.
request = json.dumps({
    "taskType": "OUTPAINTING",
    "outPaintingParams": {
        "image": image_to_base64(extended_image),
        "maskImage": image_to_base64(mask_image),
        "text": "A girl standing on a grass field in a dark night with stars and a full moon.",  # Description of the background to generate
        "outPaintingMode": "DEFAULT",  # "DEFAULT" softens the mask. "PRECISE" keeps it sharp.
    },
    "imageGenerationConfig": {
        "numberOfImages": 1,  # Number of variations to generate
        "quality": "premium",  # Allowed values are "standard" or "premium"
        "width": target_width,
        "height": target_height,
        "cfgScale": 8,
        "seed": 5763,  # Use a random seed
    },
})

modelId = "amazon.titan-image-generator-v1"

response = bedrock_runtime.invoke_model(body=request, modelId=modelId)
response_body = json.loads(response.get("body").read())

image_bytes = base64.b64decode(response_body["images"][0])
outpaint = Image.open(io.BytesIO(image_bytes))

make_image_grid([extended_image, outpaint], rows=1, cols=2)